**Criando conexão no sistema.**

In [ ]:
from google.cloud.dataproc_spark_connect import DataprocSparkSession
from google.cloud.dataproc_v1 import Session
from pyspark.sql import functions as F
from pyspark.sql.functions import explode, col


# This will create a default Spark session
spark = DataprocSparkSession.builder.getOrCreate()

█████████████████████████████████████████████████████████▌                      

**Lendo tabela da bronze**

In [ ]:
df = spark.read.format("bigquery").option("table", "clinic-de.1_bronze.orders").load()

df.printSchema()


root
 |-- metadata: struct (nullable = true)
 |    |-- channel: string (nullable = true)
 |-- totals: struct (nullable = true)
 |    |-- amount: double (nullable = true)
 |    |-- items: long (nullable = true)
 |-- diagnosis: string (nullable = true)
 |-- status: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- test_name: string (nullable = true)
 |    |    |-- instructions: string (nullable = true)
 |    |    |-- price: double (nullable = true)
 |    |    |-- test_code: string (nullable = true)
 |    |    |-- order_item_id: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- priority: string (nullable = true)
 |-- physician_id: string (nullable = true)
 |-- fasting: boolean (nullable = true)
 |-- patient_id: string (nullable = true)
 |-- order_id: string (nullable = true)



**Selecionando colunas necessarias do dataframe.**

In [ ]:
df_tratado_v1 = (df.select(
    "items",
    "order_id"

)
)

**Tratamento nos dados com exploded e seleção de colunas necessarias.**

In [ ]:
df_exploded = df_tratado_v1.withColumn("item", explode("items"))

In [ ]:
df_tratado_v2 = df_exploded.select(
    "item.test_name",
    "item.instructions",
    "item.price",
    "item.test_code",
    "item.order_item_id",
    "order_id"

)

**Salvando dataframe na silver.**

In [ ]:
(df_tratado_v2.write
        .format("bigquery")
        .option("table", "clinic-de.2_silver.orders_items")
        .option("writeMethod", "direct")
        .mode("overwrite")
        .save()
    )

print("----------------------Tabela salva com sucesso----------------------")

----------------------Tabela salva com sucesso----------------------


In [ ]:
spark.stop()